# Proposal 3

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm
from itertools import product

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    forgetting_metrics,
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUNV1

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 10
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"retraining_saturation"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_3.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_e': 10.0,
    'lambda_f': 10.0,
    'eps': 0.0001
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MWUNV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
training_exps = product(range(5), train_stream)

for (_, experience) in tqdm(training_exps):
    strategy.train(experience)
    strategy.eval(test_stream)

0it [00:00, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.93it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 117.88it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 118.72it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:03<00:00, 118.97it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 115.54it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 146.08it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9192
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 148.19it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1218
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 125.52it/s]
>

1it [00:25, 25.59s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1483
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1946
-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 118.49it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 117.88it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 118.83it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 113.33it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:03<00:00, 118.42it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 147.65it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0210
	ExperienceForgetting/eval_phase/test_st

2it [00:51, 25.51s/it]

...


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.0518
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.0518
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8953
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0227
	StreamForgetting/eval_phase/test_stream = 0.0227
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9288
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.01it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.88it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 113.07it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 114.37it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 114.92it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 141.35it/s]
> Eval on experience 0 (Ta

49it [21:13, 26.37s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.0569
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.0569
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8902
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0217
	StreamForgetting/eval_phase/test_stream = 0.0217
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9301
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 114.34it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 110.12it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 114.28it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 113.04it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 113.78it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 142.14it/s]
> Eval on experience 0 (Ta

50it [21:39, 25.99s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.9514
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0206
	StreamForgetting/eval_phase/test_stream = 0.0206
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9315


# Get metrics

In [12]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [13]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))